In [ ]:
import pydicom
import SimpleITK as sitk
from rt_utils import RTStructBuilder

# Paths to DICOM folders (not NIfTI!)
ct_folder = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\full_CT"
rs_file = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\RS_file.dcm"

ct_image = sitk.ReadImage(ct_folder)

rtstruct = RTStructBuilder.create_from(
    dicom_series_path=ct_folder,
    rt_struct_path=rs_file
)

print(rtstruct.get_roi_names())

# Example: get a specific ROI mask (as numpy array)
mask = rtstruct.get_roi_mask_by_name("Brainstem")

# Convert mask to SimpleITK image (same space as CT)
mask_image = sitk.GetImageFromArray(mask.astype('uint8'))
mask_image.CopyInformation(ct_image)

# Save mask as NIfTI
sitk.WriteImage(mask_image, "manual_brainstem.nii.gz")

In [3]:
from pathlib import Path
import pydicom
import numpy as np
import SimpleITK as sitk
from skimage.draw import polygon   # make sure this is installed!

# Define the function (paste your code here)

def voxelise_rtstruct(rt_path, ct_dir, out_dir, z_offset=0, new_z_spacing=1.0):
    # --- Load CT volume ---
    reader = sitk.ImageSeriesReader()
    series_files = reader.GetGDCMSeriesFileNames(str(ct_dir))
    reader.SetFileNames(series_files)
    ct_img = reader.Execute()

    # --- Extract geometry ---
    spacing = np.array(ct_img.GetSpacing())        # (sx, sy, sz)
    origin = np.array(ct_img.GetOrigin())          # (ox, oy, oz)
    direction = np.array(ct_img.GetDirection())    # 3x3 row-major
    size = np.array(ct_img.GetSize())[::-1]        # (z,y,x) for array
    z_positions = np.arange(size[0]) * spacing[2] + origin[2]

    # --- Load RTSTRUCT ---
    rt = pydicom.dcmread(str(rt_path))
    print(f"Processing {Path(rt_path).name}...")
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    for roi, roi_contour in zip(rt.StructureSetROISequence, rt.ROIContourSequence):
        name = roi.ROIName.replace(" ", "_")
        mask = np.zeros(size, dtype=bool)
        for contour in roi_contour.ContourSequence:
            pts = np.array(contour.ContourData).reshape(-1, 3)
            z = np.mean(pts[:, 2])
            slice_idx = np.argmin(np.abs(z_positions - z)) + z_offset
            slice_idx = max(0, min(slice_idx, mask.shape[0] - 1))
            R = direction.reshape(3, 3)
            ijk = ((pts - origin) / spacing) @ np.linalg.inv(R).T
            x = ijk[:, 0]
            y = ijk[:, 1]
            rr, cc = polygon(y, x, shape=mask.shape[1:])
            mask[slice_idx, rr, cc] = True
        mask_img = sitk.GetImageFromArray(mask.astype(np.uint8))
        mask_img.CopyInformation(ct_img)
        out_path = Path(out_dir) / f"{name}.mha"
        sitk.WriteImage(mask_img, str(out_path))
        print(f"  → saved {out_path.name}")
    print("Done.\n")

# --- Define your inputs ---
rt_path = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\RS_file\RS1.2.752.243.1.1.20230830173944533.9000.76237.dcm")
ct_dir  = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\full_CT")
out_dir = Path(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations")

# --- Run it ---
voxelise_rtstruct(rt_path, ct_dir, out_dir)


Processing RS1.2.752.243.1.1.20230830173944533.9000.76237.dcm...
  → saved Dentition.mha
  → saved Orbit_R.mha
  → saved Orbit_L.mha
  → saved Nasal_bone.mha
  → saved Sphenoid_R.mha
  → saved Sphenoid_L.mha
  → saved Ethmoid.mha
  → saved Mandible_R.mha
  → saved Mandible_L.mha
  → saved Maxilla_R.mha
  → saved Maxilla_L.mha
  → saved TMJ_R.mha
  → saved TMJ_L.mha
  → saved External.mha
  → saved Dentition_(1).mha
  → saved Orbit_R_(1).mha
  → saved Orbit_L_(1).mha
  → saved Nasal_bone_(1).mha
  → saved Sphenoid_R_(1).mha
  → saved Sphenoid_L_(1).mha
  → saved Ethmoid_(1).mha
  → saved Mandible_R_(1).mha
  → saved Mandible_L_(1).mha
  → saved Maxilla_R_(1).mha
  → saved Maxilla_L_(1).mha
  → saved TMJ_R_(1).mha
  → saved TMJ_L_(1).mha
  → saved M1.mha
  → saved I1.mha
  → saved Isocenter_1.mha
  → saved Isocenter_2.mha
Done.



In [2]:
import SimpleITK as sitk
from rt_utils import RTStructBuilder

ct_folder = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\full_CT"
rs_file = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\RS_file\RS1.2.752.243.1.1.20230830173944533.9000.76237.dcm"
out_dir   = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\manual_segmentations"

# Load CT
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(ct_folder)
reader.SetFileNames(dicom_names)
ct_image = reader.Execute()

# Load RTSTRUCT
rtstruct = RTStructBuilder.create_from(
    dicom_series_path=ct_folder,
    rt_struct_path=rs_file
)

# Export all ROIs
roi_names = rtstruct.get_roi_names()
for i, name in enumerate(roi_names):
    if not name or name.strip() == "":
        name = f"ROI_{i+1}"
    mask = rtstruct.get_roi_mask_by_name(name)
    mask_image = sitk.GetImageFromArray(mask.astype('uint8'))
    mask_image.CopyInformation(ct_image)
    out_path = f"{out_dir}\\manual_{name}.nii.gz"
    sitk.WriteImage(mask_image, out_path)
    print(f"Saved: {out_path}")


RuntimeError: Exception thrown in SimpleITK Image_CopyInformation: D:\a\SimpleITK\SimpleITK\Code\Common\src\sitkImage.cxx:353:
sitk::ERROR: Source image size of [ 512, 512, 245 ] does not match this image's size of [ 245, 512, 512 ]!